# 2014 - 2018 Power Plant Python Loading
## Prior to 2014 has not been developed yet based on different column headers

In [1]:
# IMPORT CSV and Pandas
import csv
import pandas as pd
import numpy as np
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile
import json
import time

reader[15].columns.str.strip().str.lower().str.replace(r'\n', ' ').str.replace('(', '').str.replace(')', '')

reader[0].columns = reader[0].columns.str.strip().str.lower().str.replace(r'\n', ' ').str.replace('(', '').str.replace(')', '')
reader[0].columns

reader[16].head(1)

In [3]:
#2018 has one file 
#2008-2017 has three files and the file we want it always in position 2

def getState(stateNum):
    if stateNum>56:
        stateNum=56
    #state conversion json
    stateRAW ='{"1": "AL","2": "AK","4": "AZ","5": "AR","6": "CA","8": "CO","9": "CT","10": "DE","11": "DC","12": "FL","13": "GA","15": "HI","16": "ID","17": "IL","18": "IN","19": "IA","20": "KS","21": "KY","22": "LA","23": "ME","24": "MD","25": "MA","26": "MI","27": "MN","28": "MS","29": "MO","30": "MT","31": "NE","32": "NV","33": "NH","34": "NJ","35": "NM","36": "NY","37": "NC","38": "ND","39": "OH","40": "OK","41": "OR","42": "PA","44": "RI","45": "SC","46": "SD","47": "TN","48": "TX","49": "UT","50": "VT","51": "VA","53": "WA","54": "WV","55": "WI","56": "WY"}'
    stateJSON = json.loads(stateRAW)
    
    return stateJSON[str(stateNum)]


#print(str(1))
#print(stateJSON[str(1)])

reader=[]
start_year=1970
end_year=2018

total_start = time.time()

for i in range(start_year,end_year+1):
    
    start = time.time()
    print('Processing ' + str(i)+' Data...')
    
    if i >= 2018:
        url = 'https://www.eia.gov/electricity/data/eia923/xls/f923_' + str(i) +'.zip'
        name_pos = 0
        start_line = 5
        
    elif i < 2018 and i > 2010:
        url = 'https://www.eia.gov/electricity/data/eia923/archive/xls/f923_' + str(i) +'.zip'
        name_pos = 1
        start_line = 5

    elif i < 2011 and i > 2007:
        url = 'https://www.eia.gov/electricity/data/eia923/archive/xls/f923_' + str(i) +'.zip'
        name_pos = 1
        start_line = 7

    elif i <= 2007 and i > 2003:
        url = 'https://www.eia.gov/electricity/data/eia923/archive/xls/f906920_' + str(i) +'.zip'
        name_pos = 1
        start_line = 7
        
    elif i>=2001 and i <= 2003:
        url = 'https://www.eia.gov/electricity/data/eia923/archive/xls/f906920_' + str(i) +'.zip'
        name_pos = 0
        start_line = 7
    elif i<2001:
        url= 'https://www.eia.gov/electricity/data/eia923/archive/xls/utility/f759'+str(i)+'u.xls'
        name_pos=0
        start_line=1
        
    if i>=2001:    
        zipped_file = ZipFile(BytesIO(urlopen(url).read()))
        reader.append(pd.read_excel(zipped_file.open(zipped_file.namelist()[name_pos]), \
                                sheet_name = 0, skiprows=start_line))
    else:
        zipped_file=urlopen(url)
        reader.append(pd.read_excel(zipped_file))
    

    # Clean up columns
    reader[i-start_year].columns = reader[i-start_year].columns.str.lower().str.replace(r'\n', ' ').str.replace('(', '').str.replace(')', '')
    
    # Update year to be 4 digit
    if i<=2000:
        reader[i-start_year]["year"]=i
        
    # Combine monthly generation to total for 1970 - 1995
    if i<1996:
        reader[i-start_year]['netgen']= reader[i-start_year]['gen01']+ \
                                        reader[i-start_year]['gen02']+ \
                                        reader[i-start_year]['gen03']+ \
                                        reader[i-start_year]['gen04']+ \
                                        reader[i-start_year]['gen05']+ \
                                        reader[i-start_year]['gen06']+ \
                                        reader[i-start_year]['gen07']+ \
                                        reader[i-start_year]['gen08']+ \
                                        reader[i-start_year]['gen09']+ \
                                        reader[i-start_year]['gen10']+ \
                                        reader[i-start_year]['gen11']+ \
                                        reader[i-start_year]['gen12']
    
    #Convert State Number to State Abbreviation for 1970-2000
    if i<2001:
        reader[i-start_year]['state']=reader[i-start_year]['fipst'].apply(getState)
    #    for j in range(0,len(reader[i-start_year])):
    #        state=reader[i-start_year]['fipst'][j]
    #        reader[i-start_year]['fipst'][j]=stateJSON[str(int(state))]
    
    
    # Get the column names to match for different version of the 923 EIA data
    if i >= 2012:
        reader[i-start_year].rename(columns={'aer fuel type code':'fueltype','net generation megawatthours': 'netgen', 'plant state': 'state'}, inplace=True)
    #elif i == 2012 or i == 2013 or i == 2014:
     #   reader[i-start_year].rename(columns={'aer fuel type code':'fueltype','net generation megawatthours': 'netgen', 'plant state': 'state'}, inplace=True)
    elif i<2012 and i>=2001:
        reader[i-start_year].rename(columns={'aer fuel type code':'fueltype','net generation megawatthours': 'netgen'}, inplace=True)
    elif i<2001 and i>=1996:
        reader[i-start_year].rename(columns={'fueldesc':'fueltype','netgenerat': 'netgen', 'pltcode':'plant id', 'pltname':'plant name'}, inplace=True)
    elif i<1996:
        reader[i-start_year].rename(columns={'fueldesc':'fueltype', 'pltcode':'plant id', 'pltname':'plant name'}, inplace=True)


    # Keep only yhe needed headers
    reader[i-start_year]=reader[i-start_year][['plant id','plant name','state', 'netgen','fueltype','year']]
    end = time.time()
    print('Processing took ' + str(end - start) + ' seconds')
    
total_end = time.time()
print('All Data Processed Successfully in ' + str(total_end - total_start)  + ' seconds')


Processing 1970 Data...
Processing took 5.384181976318359 seconds
Processing 1971 Data...
Processing took 4.30669903755188 seconds
Processing 1972 Data...
Processing took 4.439879894256592 seconds
Processing 1973 Data...
Processing took 4.448336839675903 seconds
Processing 1974 Data...
Processing took 4.838825941085815 seconds
Processing 1975 Data...
Processing took 4.5097739696502686 seconds
Processing 1976 Data...
Processing took 4.664414882659912 seconds
Processing 1977 Data...
Processing took 4.012689828872681 seconds
Processing 1978 Data...
Processing took 3.9866130352020264 seconds
Processing 1979 Data...
Processing took 4.014747142791748 seconds
Processing 1980 Data...
Processing took 3.78092885017395 seconds
Processing 1981 Data...
Processing took 4.009830951690674 seconds
Processing 1982 Data...
Processing took 4.146789789199829 seconds
Processing 1983 Data...
Processing took 4.458010911941528 seconds
Processing 1984 Data...
Processing took 3.9223077297210693 seconds
Processin

In [95]:
#Concatenate
data = pd.concat(reader)
print('Individual Dataframes concatenated.')
#This removes the non-power plant data at the end
data =data[~data['plant id'].isin([99999])]



#this removes plant names that are empty
data = data[~data['plant name'].isna()]
print(data.shape)
data.head(10)

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


Individual Dataframes concatenated.
(351388, 7)


,fueltype,lat,netgen,plant id,plant name,state,year
3,WAT,NaN,13208900.0,510.0,LEWISTON,ME,1970
4,WAT,NaN,0.0,510.0,LEWISTON,ME,1970
5,WAT,NaN,0.0,360.0,WATER PWR,ME,1970
6,WAT,NaN,32040300.0,360.0,WATER PWR,ME,1970
7,WAT,NaN,122793000.0,622.0,MADAWASKA,ME,1970
8,WAT,NaN,525690357.0,96.0,PENOBSCOT M,ME,1970
9,WAT,NaN,468326160.0,96.0,PENOBSCOT M,ME,1970
10,WAT,NaN,403657000.0,990.0,ANDRSCOGGIN,ME,1970
11,WAT,NaN,24022500.0,22.0,OTIS,ME,1970
12,WAT,NaN,18690000.0,22.0,OTIS,ME,1970


In [15]:
#Get the EIA 860m Plant Data. This Contains Latitude & Longitude Data
url = 'https://www.eia.gov/electricity/data/eia860m/xls/december_generator2018.xlsx'
response = urlopen(url)
df_plant_data = pd.read_excel(response, skipinitialspace = True, quotechar = '"')
new_header = df_plant_data.iloc[0] #grab the first row for the header
df_plant_data = df_plant_data[1:] #take the data less the header row
df_plant_data.columns = new_header #set the h

#drop duplicats for plant_id
df_plant_data.drop_duplicates(subset="Plant ID")
df_plant_data.head(5)        


,Entity ID,Entity Name,Plant ID,Plant Name,Sector,Plant State,Generator ID,Unit Code,Nameplate Capacity (MW),Net Summer Capacity (MW),...,Planned Derate of Summer Capacity (MW),Planned Uprate Year,Planned Uprate Month,Planned Uprate of Summer Capacity (MW),County,Latitude,Longitude,Google Map,Bing Map,Balancing Authority Code
1,195,Alabama Power Co,2,Bankhead Dam,Electric Utility,AL,1,NaN,53.9,53,...,,,,,Tuscaloosa,33.4587,-87.3568,Map,Map,SOCO
2,195,Alabama Power Co,3,Barry,Electric Utility,AL,1,NaN,153.1,55,...,,,,,Mobile,31.0069,-88.0103,Map,Map,SOCO
3,195,Alabama Power Co,3,Barry,Electric Utility,AL,2,NaN,153.1,55,...,,,,,Mobile,31.0069,-88.0103,Map,Map,SOCO
4,195,Alabama Power Co,3,Barry,Electric Utility,AL,4,NaN,403.7,362,...,,,,,Mobile,31.0069,-88.0103,Map,Map,SOCO
5,195,Alabama Power Co,3,Barry,Electric Utility,AL,5,NaN,788.8,756.5,...,,,,,Mobile,31.0069,-88.0103,Map,Map,SOCO


In [19]:
#drop duplicats for plant_id
df_plant_data.drop_duplicates(subset="Plant ID",keep = 'first', inplace = True)
df_plant_data.head(5)        

,Entity ID,Entity Name,Plant ID,Plant Name,Sector,Plant State,Generator ID,Unit Code,Nameplate Capacity (MW),Net Summer Capacity (MW),...,Planned Derate of Summer Capacity (MW),Planned Uprate Year,Planned Uprate Month,Planned Uprate of Summer Capacity (MW),County,Latitude,Longitude,Google Map,Bing Map,Balancing Authority Code
1,195,Alabama Power Co,2,Bankhead Dam,Electric Utility,AL,1,NaN,53.9,53,...,,,,,Tuscaloosa,33.4587,-87.3568,Map,Map,SOCO
2,195,Alabama Power Co,3,Barry,Electric Utility,AL,1,NaN,153.1,55,...,,,,,Mobile,31.0069,-88.0103,Map,Map,SOCO
12,195,Alabama Power Co,4,Walter Bouldin Dam,Electric Utility,AL,1,NaN,75,74.7,...,,,,,Elmore,32.5839,-86.2831,Map,Map,SOCO
15,195,Alabama Power Co,7,Gadsden,Electric Utility,AL,1,NaN,69,64,...,,,,,Etowah,34.0128,-85.9708,Map,Map,SOCO
17,195,Alabama Power Co,8,Gorgas,Electric Utility,AL,10,NaN,788.8,727.7,...,,,,,Walker,33.6443,-87.1965,Map,Map,SOCO


In [20]:
df_plant_data

,Entity ID,Entity Name,Plant ID,Plant Name,Sector,Plant State,Generator ID,Unit Code,Nameplate Capacity (MW),Net Summer Capacity (MW),...,Planned Derate of Summer Capacity (MW),Planned Uprate Year,Planned Uprate Month,Planned Uprate of Summer Capacity (MW),County,Latitude,Longitude,Google Map,Bing Map,Balancing Authority Code
1,195,Alabama Power Co,2,Bankhead Dam,Electric Utility,AL,1,NaN,53.9,53,...,,,,,Tuscaloosa,33.4587,-87.3568,Map,Map,SOCO
2,195,Alabama Power Co,3,Barry,Electric Utility,AL,1,NaN,153.1,55,...,,,,,Mobile,31.0069,-88.0103,Map,Map,SOCO
12,195,Alabama Power Co,4,Walter Bouldin Dam,Electric Utility,AL,1,NaN,75,74.7,...,,,,,Elmore,32.5839,-86.2831,Map,Map,SOCO
15,195,Alabama Power Co,7,Gadsden,Electric Utility,AL,1,NaN,69,64,...,,,,,Etowah,34.0128,-85.9708,Map,Map,SOCO
17,195,Alabama Power Co,8,Gorgas,Electric Utility,AL,10,NaN,788.8,727.7,...,,,,,Walker,33.6443,-87.1965,Map,Map,SOCO
20,5701,El Paso Electric Co,9,Copper,Electric Utility,TX,1,NaN,80.5,62,...,,,,,El Paso,31.7569,-106.375,Map,Map,EPE
21,195,Alabama Power Co,10,Greene County,Electric Utility,AL,1,NaN,299.2,257.8,...,,,,,Greene,32.6017,-87.7811,Map,Map,SOCO
32,195,Alabama Power Co,11,H Neely Henry Dam,Electric Utility,AL,1,NaN,24.3,23.7,...,,,,,Calhoun,33.7845,-86.0524,Map,Map,SOCO
35,195,Alabama Power Co,12,Holt Dam,Electric Utility,AL,1,NaN,46.9,48,...,,,,,Tuscaloosa,33.2553,-87.4495,Map,Map,SOCO
36,195,Alabama Power Co,13,Jordan Dam,Electric Utility,AL,1,NaN,25,32.3,...,,,,,Elmore,32.6189,-86.2548,Map,Map,SOCO


In [121]:
def findLat(plantCode):
    try:
        #lat=float(df_plant_data.loc[(df_plant_data['Plant ID']==plantCode) & (df_plant_data['Plant Name']==plantName)]['Latitude'].values)
        #long=float(df_plant_data.loc[(df_plant_data['Plant ID']==plantCode) & (df_plant_data['Plant Name']==plantName)]['Longitude'].values)
        #name=(df_plant_data.loc[df_plant_data['Plant ID']==plantCode]['Plant Name'].values)[0]
        
        lat=float(df_plant_data.loc[(df_plant_data['Plant ID']==plantCode)]['Latitude'].values)
        long=float(df_plant_data.loc[(df_plant_data['Plant ID']==plantCode)]['Longitude'].values)
        name=(df_plant_data.loc[df_plant_data['Plant ID']==plantCode]['Plant Name'].values)[0]
    #name=str(1)
    
        return [lat,long,name]
    except:
        if plantCode!=99999:
            print(plantCode)
        return [None,None,None]

print(findLat(3))

[31.0069, -88.0103, 'Barry']


In [122]:
reader[48]['latlong']=reader[48]['plant id'].apply(findLat)

207
207
207
302
302
302
331
345
345
1048
1048
1552
1552
1552
1552
2080
2080
2094
2094
2094
2094
2388
2850
2850
2850
3497
3497
3497
6031
6031
6031
6147
6147
6147
6170
6170
6170
6170
6648
6648
8809
8812
8812
8816
8816
8816
8823
8823
8824
8825
8827
8828
8829
8831
8832
8834
8834
8835
8835
8837
8837
8838
8838
8841
8841
8841
8843
8843
8845
8845
8846
8846
8847
8847
8848
8850
8851
8851
8852
8852
8853
8853
8857
8858
8865
8866
8866
8867
8867
8868
8899
8899
10090
10090
10090
10333
10333
10333
10810
10810
10811
10811
10812
10812
52071
52071
54534
54561
54561
54779
54779
55526
55867
55867
55867
57759
57759
57759
57759


In [123]:
reader[48].head(100)

,plant id,plant name,state,netgen,fueltype,year,lat,latlong
0,3,Barry,AL,2345699.000,NG,2018,31.006900,"[31.0069, -88.0103, Barry]"
1,3,Barry,AL,4341821.000,NG,2018,31.006900,"[31.0069, -88.0103, Barry]"
2,3,Barry,AL,0.000,COL,2018,31.006900,"[31.0069, -88.0103, Barry]"
3,3,Barry,AL,3227353.000,COL,2018,31.006900,"[31.0069, -88.0103, Barry]"
4,3,Barry,AL,143099.030,NG,2018,31.006900,"[31.0069, -88.0103, Barry]"
5,3,Barry,AL,406943.960,COL,2018,31.006900,"[31.0069, -88.0103, Barry]"
6,7,Gadsden,AL,188877.000,NG,2018,34.012800,"[34.0128, -85.9708, Gadsden]"
7,8,Gorgas,AL,5490646.400,COL,2018,33.644344,"[33.644344, -87.19649, Gorgas]"
8,8,Gorgas,AL,7241.585,DFO,2018,33.644344,"[33.644344, -87.19649, Gorgas]"
9,10,Greene County,AL,10825.873,DFO,2018,32.601700,"[32.6017, -87.7811, Greene County]"
